In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import ast
import numpy as np
from irt.evaluation import eval_utils
from irt.data import data_loader
import matplotlib.pyplot as plt
import time
import girth
# from girth import rasch_conditional
from irt.data.rasch import generate_data
from irt.algorithms.spectral_estimator import spectral_estimate
from irt.algorithms import conditional_mle
from irt.algorithms import rasch_mml
from irt.evaluation.eval_utils import log_likelihood_heldout, bayesian_auc, pairwise_disagreement_error, top_k_accuracy
import warnings
warnings.filterwarnings("ignore")

In [26]:
cutoff = 25
A_lsat = data_loader.lsat()
A_ml_100k, ml_100k_ratings = data_loader.ml_100k(cutoff=50)
A_hetrec_2k, hetrec_2k_ratings  = data_loader.hetrec_2k(cutoff=50)
A_ml_1m, ml_1m_ratings = data_loader.ml_1m(cutoff=50)

A_ml_20m, ml_20m_ratings = data_loader.ml_20m(cutoff=50)
A_bx, bx_ratings = data_loader.bx_book(cutoff=50)
A_ml_10m, ml_10m_ratings = data_loader.ml_10m(cutoff=cutoff)
A_each, each_ratings = data_loader.each_movie(cutoff=cutoff)
A_jester, jester_ratings = data_loader.jester(cutoff=cutoff)
A_genome, genome_ratings = data_loader.book_genome(cutoff=cutoff)


In [22]:
A_bx, bx_ratings = data_loader.bx_book(cutoff=100)

In [32]:
ratings = each_ratings
all_num_ratings = [num_ratings for (_, _, num_ratings) in ratings][:200]
print(np.mean(all_num_ratings))
print(np.max(all_num_ratings))
print(np.median(all_num_ratings))
print(np.min(all_num_ratings))
print(A_bx.shape)

6971.165
32864
4194.5
63
(731, 105283)


In [24]:
sorted_ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
true_rank = [int(item) for (item, _, _) in sorted_ratings] # Sort from most popular items
sorted_ratings[:50]



[(450, 6.6117647058823525, 170),
 (178, 6.541237113402062, 194),
 (277, 6.467005076142132, 197),
 (424, 6.363095238095238, 168),
 (582, 5.57185628742515, 334),
 (283, 5.345132743362832, 226),
 (410, 5.302564102564102, 195),
 (588, 5.194174757281553, 103),
 (120, 5.103626943005182, 193),
 (385, 5.00711743772242, 281),
 (266, 4.920308483290489, 389),
 (239, 4.915730337078652, 178),
 (612, 4.907407407407407, 162),
 (131, 4.900175131348512, 571),
 (149, 4.874015748031496, 127),
 (577, 4.820143884892087, 139),
 (327, 4.782945736434108, 129),
 (444, 4.756410256410256, 156),
 (107, 4.72934472934473, 351),
 (122, 4.695121951219512, 164),
 (286, 4.6923076923076925, 104),
 (632, 4.654867256637168, 113),
 (432, 4.652321630804077, 883),
 (418, 4.635235732009925, 403),
 (90, 4.6328125, 128),
 (596, 4.615384615384615, 130),
 (124, 4.614583333333333, 192),
 (535, 4.609589041095891, 146),
 (76, 4.554621848739496, 119),
 (116, 4.553072625698324, 179),
 (623, 4.550632911392405, 158),
 (381, 4.5496183206

In [18]:

for ratings in [
        ml_100k_ratings,
        hetrec_2k_ratings,
        ml_1m_ratings,
        ml_10m_ratings,
        ml_20m_ratings,
        bx_ratings,
        each_ratings,
        jester_ratings,
        genome_ratings,
    ]:

    sorted_ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
    true_rank = [int(item) for (item, _, _) in sorted_ratings] # Sort from most popular items
    print("top ratings min number of ratings", np.min([num_rating for (_, _, num_rating) in sorted_ratings[:50]]))
    print()



top ratings min number of ratings 25

top ratings min number of ratings 50

top ratings min number of ratings 56

top ratings min number of ratings 95

top ratings min number of ratings 402

top ratings min number of ratings 52

top ratings min number of ratings 123

top ratings min number of ratings 8616

top ratings min number of ratings 159



In [17]:
print("A_ml_1m", A_ml_1m.shape)
print("A_ml_100k", A_ml_100k.shape)
print("A_hetrec_2k", A_hetrec_2k.shape)
print("A_ml_10m", A_ml_10m.shape)
print("A_jester", A_jester.shape)
print("A_ml_20m", A_ml_20m.shape)
print("A_each", A_each.shape)
print("A_bx", A_bx.shape)
print("A_genome", A_genome.shape)


A_ml_1m (2514, 6040)
A_ml_100k (1050, 610)
A_hetrec_2k (3261, 2113)
A_ml_10m (8653, 69878)
A_jester (100, 24982)
A_ml_20m (10524, 138493)
A_each (1474, 61265)
A_bx (2185, 105283)
A_genome (9374, 350332)


In [15]:
A_ml_100k, ml_100k_ratings = data_loader.ml_100k(cutoff=25)
print(A_ml_100k.shape)


(1050, 610)


In [20]:
ratings = each_ratings
sorted_ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
true_rank = [int(item) for (item, _, _) in sorted_ratings] # Sort from most popular items
sorted_ratings[:40]

[(496, 5.392391304347826, 12880),
 (1062, 5.38662486938349, 957),
 (302, 5.337928937097674, 19729),
 (851, 5.328055731475618, 1579),
 (1100, 5.318230088495575, 2825),
 (843, 5.289975144987572, 1207),
 (1095, 5.248381400824014, 1699),
 (847, 5.2476290832455215, 949),
 (862, 5.236363636363636, 1155),
 (1098, 5.229255493604461, 3049),
 (798, 5.227263395606651, 4871),
 (1105, 5.1875, 624),
 (1109, 5.1819066147859925, 1028),
 (46, 5.156688218994038, 11909),
 (1125, 5.144702842377261, 387),
 (842, 5.138193688792166, 919),
 (667, 5.1298527443105755, 1494),
 (1099, 5.124936772888215, 1977),
 (551, 5.118464653902798, 21728),
 (1106, 5.11264367816092, 870),
 (852, 5.112418300653594, 765),
 (695, 5.111358574610245, 1347),
 (1048, 5.107784431137724, 334),
 (1114, 5.101333333333334, 375),
 (244, 5.099838845829177, 13031),
 (861, 5.0807692307692305, 260),
 (1077, 5.07492795389049, 694),
 (864, 5.071969696969697, 264),
 (1135, 5.069090909090909, 1375),
 (900, 5.066125290023201, 862),
 (1123, 5.065985

In [32]:
A_lsat = data_loader.lsat()
A_uci = data_loader.uci_student()
A_three = data_loader.grades_three()

In [33]:
print(A_lsat.shape)
print(A_uci.shape)
print(A_three.shape)

(5, 1000)
(4, 131)
(3, 649)


In [35]:
A_three[:, :20]

array([[0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0]])

In [36]:
A_uci[:, :20]

array([[0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1]])

In [55]:
f = open("../irt/data/datasets/algebra_2005_2006_master.txt")

In [56]:
header = f.readline().rstrip().split("\t")

In [45]:
for line in f.readlines():
    data = line.rstrip().split("\t")
    

In [47]:
header

['Row',
 'Anon Student Id',
 'Problem Hierarchy',
 'Problem Name',
 'Problem View',
 'Step Name',
 'Step Start Time',
 'First Transaction Time',
 'Correct Transaction Time',
 'Step End Time',
 'Step Duration (sec)',
 'Correct Step Duration (sec)',
 'Error Step Duration (sec)',
 'Correct First Attempt',
 'Incorrects',
 'Hints',
 'Corrects',
 'KC(Default)',
 'Opportunity(Default)']

In [46]:
line.rstrip().split("\t")

['390',
 '0BrbPbwCMz',
 'Unit ES_04, Section ES_04-15',
 'LIT59',
 '1',
 'b+r*(x+y) = v-s',
 '2005-10-05 12:45:31.0',
 '2005-10-05 12:45:46.0',
 '2005-10-05 12:45:46.0',
 '2005-10-05 12:45:46.0',
 '15',
 '15',
 '',
 '1',
 '0',
 '0',
 '1',
 '[SkillRule: Remove constant; {ax+b=c, positive; ax+b=c, negative; x+a=b, positive; x+a=b, negative; [var expr]+[const expr]=[const expr], positive; [var expr]+[const expr]=[const expr], negative; [var expr]+[const expr]=[const expr], all; Combine constants to right; Combine constants to left; a-x=b, positive; a/x+b=c, positive; a/x+b=c, negative}]',
 '77']

In [57]:
A, problems = data_loader.algebra_05_06()

In [51]:
algebra.shape

(686, 575)

In [59]:
np.sum(algebra == 1, 1)

array([ 2,  0,  1,  1,  1,  3, 10,  4,  1, 26,  4,  1,  6,  5,  9, 22,  1,
        1,  4,  6,  0,  2,  9,  0, 18,  2,  1,  0,  1,  3,  2,  1, 15,  1,
        3,  7,  7,  0,  1,  2,  1,  2,  1,  0,  1,  3,  2,  3,  1, 24,  1,
        1,  3,  1,  1,  2,  5,  0,  0,  2,  2,  1,  2,  9,  8,  1,  3,  0,
        5,  1,  2,  4,  2,  3,  1,  1,  1,  2,  1,  1, 14,  4,  0,  3,  9,
        3,  4,  1,  0,  2,  2,  5,  7,  7,  3,  5,  0,  4,  0,  0,  2,  2,
       19,  1,  3, 14,  1,  1, 13,  1,  2,  1,  3,  1, 14,  2,  4,  2,  3,
        4,  5,  0,  2,  8,  1, 17,  1,  5, 10,  0,  2,  3,  2,  1,  6,  1,
        9,  1,  3,  3,  4,  2,  1,  1,  2,  1,  3,  3, 12,  9,  2,  3, 32,
        1,  4,  2,  3,  7,  0,  3,  4,  5,  5,  0, 14,  5,  1, 16,  0,  1,
        4,  0,  5,  2,  1,  2, 26,  2,  1,  4,  0,  2,  1,  1, 11,  9,  1,
        0,  1,  2,  4,  6,  2,  1,  6,  2,  2,  1,  3,  2,  1,  1,  1,  2,
       18,  5,  2,  0, 10,  4,  2,  5,  4,  3,  0,  1,  0,  1,  1,  1,  2,
        1,  2, 10, 24, 14

In [58]:
problems

{'1PTB02',
 '1PTB03',
 '1PTB05',
 '1PTB06',
 '1PTB09',
 '1PTFB10',
 '1PTFB11',
 '1PTFB12',
 '1PTFB13',
 '1PTFB14',
 '1PTFB15',
 '1PTFB16',
 '1PTFB17',
 '1PTFB18',
 '2PTFB10',
 '2PTFB12',
 '2PTFB13',
 '2PTFB14',
 '2PTFB15',
 '2PTFB16',
 '2PTFB18',
 '2PTFB19',
 '2PTFB20',
 '2PTFB21',
 '2PTFB22',
 '2PTS01',
 '2PTS02',
 '2PTS03',
 '2PTS07',
 '2PTS08',
 '2X+3YGT9',
 '4X-5YGT40',
 '7X-4YGE15',
 'BH1T02',
 'BH1T02B',
 'BH1T03',
 'BH1T04',
 'BH1T05',
 'BH1T06',
 'BH1T07',
 'BH1T08C',
 'BH1T10A',
 'BH1T10C',
 'BH1T13',
 'BH1T15B',
 'BH1T17',
 'BH1T18',
 'BH1T19',
 'BH1T19A',
 'BH1T19B',
 'BH1T19C',
 'BH1T20',
 'BH1T21',
 'BH1T21A',
 'BH1T21C',
 'BH1T22A',
 'BH1T22C',
 'BH1T23',
 'BH1T23A',
 'BH1T24',
 'BH1T24A',
 'BH1T24B',
 'BH1T26C',
 'BH1T27',
 'BH1T27A',
 'BH1T27B',
 'BH1T27C',
 'BH1T28',
 'BH1T28A',
 'BH1T28B',
 'BH1T28C',
 'BH1T29',
 'BH1T29A',
 'BH1T29B',
 'BH1T29C',
 'BH1T30',
 'BH1T30B',
 'BH1T30C',
 'BH1T31',
 'BH1T31A',
 'BH1T32',
 'BH1T32A',
 'BH1T32B',
 'BH1T32C',
 'BH1T33',
 'BH1T

In [84]:


betas = np.random.normal(0, 3, size=(100,))
sigma = 1


from scipy.integrate import quadrature
from scipy.stats import norm
from scipy.special import expit
import time



In [96]:
start = time.time()
p_response = eval_utils.quarature_p_response(betas, 2)
print(time.time() - start)

0.5120556354522705


In [95]:
start = time.time()
p_response = eval_utils.estimate_p_response(betas, 2)
print(time.time() - start)

0.60123610496521


In [86]:
x = np.random.normal(0, sigma, size=(1000000,))
start = time.time()
for beta in betas:
    p_est = np.mean(expit(x - beta))
print(time.time() - start)


1.1984446048736572
